# Evaluate Embedding Model for use in Retrieval

In [17]:
from valor import (
    Dataset, 
    Model, 
    Datum,
    GroundTruth, 
    Prediction, 
    Annotation, 
    Label, 
    Client,
    connect,
)
from valor.schemas import Embedding
from valor.enums import EvaluationStatus, MetricType

connect("http://localhost:8000")
client = Client()

Successfully connected to host at http://localhost:8000/


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from scipy.stats import skew

# embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
embedding_model = SentenceTransformer("all-mpnet-base-v2")
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
def embed(x: str) -> Embedding:
    y = embedding_model.encode(x)
    y = y.tolist()
    return Embedding(y)

# Documents

The following information is scraped from Wikipedia articles of various topics. OpenAI's GPT4o has generated short sentences and queries about the articles.

In [20]:
llama_statements = [
    "The llama (/ˈlɑːmə/; Spanish pronunciation: [ˈʎama] or [ˈʝama]) is a domesticated South American camelid.",
    "Llamas have been used by Andean cultures as meat and pack animals since the pre-Columbian era.",
    "Llamas are social animals that live in herds.",
    "Their wool is soft and contains only a small amount of lanolin.",
    "Llamas can learn simple tasks after a few repetitions.",
    "They can carry about 25 to 30% of their body weight for 8 to 13 km (5–8 miles).",
    "The name 'llama' was adopted by European settlers from native Peruvians.",
    "Llamas' ancestors originated from the Great Plains of North America about 40 million years ago.",
    "They migrated to South America about three million years ago.",
    "This migration occurred during the Great American Interchange.",
    "Camelids became extinct in North America by the end of the last ice age (10,000–12,000 years ago).",
    "As of 2007, there were over seven million llamas and alpacas in South America.",
    "The United States and Canada had over 158,000 llamas and 100,000 alpacas in 2007.",
    "These North American populations descended from llamas imported in the late 20th century.",
    "Llamas are used for carrying loads in Andean cultures.",
    "The word 'llama' was sometimes spelled 'lama' or 'glama' in the past.",
    "Llamas are capable of transporting loads over long distances.",
    "Llamas are known for their soft wool.",
    "European settlers learned about llamas from the indigenous people of Peru.",
    "The llama's migration to South America was a significant event in its evolutionary history.",
    "Llamas belong to the species Lama glama.",
    "They are widely used as pack animals in the Andes.",
    "Llamas can live and work in harsh mountainous environments.",
    "The soft wool of llamas is prized for its quality.",
    "Llamas are capable of adapting to various tasks.",
    "They play an important role in Andean agriculture and transportation.",
    "Llamas communicate using a series of vocalizations.",
    "The llama's scientific name is Lama glama.",
    "They have been domesticated for thousands of years.",
    "Llamas are closely related to alpacas, vicuñas, and guanacos.",
    "They have a lifespan of about 15 to 25 years.",
    "Llamas are known for their gentle and calm demeanor.",
    "Their ancestors once roamed North America.",
    "Llamas have a unique split-upper lip.",
    "They can thrive on a diet of grass and hay.",
    "Llamas are often used in therapy and educational programs.",
    "Their wool comes in a variety of natural colors.",
    "Llamas have been depicted in ancient Andean art and artifacts.",
    "They are an integral part of Andean cultural heritage.",
    "Llamas can form strong bonds with humans and other animals.",
    "Llamas can spit as a defense mechanism.",
    "They have padded feet that are well-suited for mountainous terrain.",
    "Llamas are known for their long necks and large eyes.",
    "They can weigh between 130 and 200 kilograms (290–440 pounds).",
    "Llamas are herbivores, primarily eating grass and hay.",
    "They have a three-compartment stomach for efficient digestion.",
    "Llamas are often used in trekking and hiking tours.",
    "They have a strong sense of curiosity.",
    "Llamas can be trained to pull carts.",
    "Their wool is hypoallergenic.",
    "Llamas have a gestation period of about 11.5 months.",
    "A baby llama is called a cria.",
    "Llamas are important to the rural economy in the Andes.",
    "They can survive with minimal water in arid environments.",
    "Llamas have a strong herding instinct.",
    "They can recognize individual humans and animals.",
    "Llamas' wool is often used for making textiles and clothing.",
    "They are sometimes used as guard animals for livestock.",
    "Llamas have been featured in various films and media.",
    "They play a significant role in traditional Andean festivals and rituals.",
]

alpaca_statements = [
    "The alpaca (Lama pacos) is a South American camelid mammal.",
    "Alpacas traditionally graze on the Andes' high plateaus.",
    "They are found in Southern Peru, Western Bolivia, Ecuador, and Northern Chile.",
    "Today, alpacas are raised globally on farms and ranches.",
    "Thousands of alpacas are born annually in North America, Europe, and Australia.",
    "There are two main breeds of alpaca: the Suri and the Huacaya.",
    "Suri alpacas produce straight 'locks' of fiber.",
    "Huacaya alpacas have crimped, wavy wool.",
    "Both types of alpaca fiber are highly valued.",
    "Alpaca fiber is used for knitted and woven items.",
    "Alpacas are similar in appearance to llamas.",
    "Alpacas are shorter than llamas.",
    "Alpacas are primarily bred for their wool.",
    "Llamas are often used as livestock guardians.",
    "Llamas also serve as pack animals.",
    "All four South American camelids are closely related.",
    "Alpacas and llamas can interbreed.",
    "Alpacas are believed to be domesticated from the vicuña.",
    "Llamas are thought to descend from the guanaco.",
    "Domestication of alpacas and llamas occurred 5,000 to 6,000 years ago.",
]

cattle_statements = [
    "Cattle (Bos taurus) are large, domesticated, bovid ungulates widely kept as livestock.",
    "They are prominent modern members of the subfamily Bovinae.",
    "Cattle are the most widespread species of the genus Bos.",
    "Mature female cattle are called cows.",
    "Mature male cattle are called bulls.",
    "Young female cattle are called heifers.",
    "Young male cattle are called oxen or bullocks.",
    "Castrated male cattle are known as steers.",
    "Cattle are commonly raised for meat.",
    "They are also raised for dairy products.",
    "Cattle are raised for leather as well.",
    "As draft animals, cattle pull carts and farm implements.",
    "In India, cattle are sacred animals within Hinduism.",
    "Cattle may not be killed in Hinduism.",
    "Small breeds like the miniature Zebu are kept as pets.",
    "Taurine cattle are widely distributed across Europe and temperate areas of Asia.",
    "Zebus are found mainly in India and tropical areas of Asia, America, and Australia.",
    "Sanga cattle are found primarily in sub-Saharan Africa.",
    "These types of cattle are sometimes classified as separate species or subspecies.",
    "There are over 1,000 recognized breeds of cattle.",
]

plastic_statements = [
    "A TPU resin consists of linear polymeric chains in block-structures.",
    "These chains contain low polarity segments which are rather long, called soft segments.",
    "Alternating with the soft segments are shorter, high polarity segments known as hard segments.",
    "Both types of segments in TPU are linked by covalent links.",
    "The segments form block-copolymers in TPU.",
    "The miscibility of the hard and soft segments depends on their glass transition temperature (Tg).",
    "Tg occurs at the onset of micro-Brownian segmental motion.",
    "It can be identified by dynamic mechanical spectra.",
    "For an immiscible TPU, the loss modulus spectrum typically shows double peaks.",
    "Each peak is assigned to the Tg of one component.",
    "If the two components are miscible, TPU shows a single broad peak.",
    "This peak's position lies between that of the two original Tg peaks.",
    "The polarity of the hard segments creates a strong attraction between them.",
    "This attraction causes a high degree of aggregation and order in this phase.",
    "Crystalline or pseudo-crystalline areas form in a soft and flexible matrix.",
    "Phase separation between blocks can vary based on polarity and molecular weight.",
    "The crystalline areas act as physical cross-links, giving TPU its high elasticity.",
    "The flexible chains provide elongation characteristics to the polymer.",
    "Pseudo cross-links disappear under heat, allowing for classical processing methods.",
    "TPU scrap can be reprocessed due to the thermal properties of the material.",
    "TPU plastic is known for its excellent elasticity.",
    "It has superior abrasion resistance.",
    "TPU is widely used in various industrial applications.",
    "This plastic offers high tensile strength.",
    "TPU can withstand low temperatures without becoming brittle.",
    "It is resistant to oil and grease.",
    "TPU plastic is often used in the production of footwear.",
    "It has a high level of transparency.",
    "TPU can be easily colored with different dyes.",
    "It is commonly used in the manufacturing of medical devices.",
    "TPU plastic is flexible and durable.",
    "It is often used to make phone cases.",
    "TPU is resistant to many chemicals.",
    "It is used in automotive parts due to its strength.",
    "TPU can be processed by injection molding.",
    "It is available in various hardness levels.",
    "TPU plastic is known for its versatility.",
    "It is used in the production of sporting goods.",
    "TPU is environmentally friendly and recyclable.",
    "This plastic has excellent impact resistance.",
]

# Questions

In [21]:
llama_questions = [
    "What is the scientific name for a llama?",
    "Where are llamas originally from?",
    "How long is the typical lifespan of a llama?",
    "What do llamas primarily eat?",
    "How much does a fully grown llama typically weigh?",
    "What is the difference between a llama and an alpaca?",
    "How many stomach compartments do llamas have?",
    "What is the average height of a llama at the shoulder?",
    "How do llamas communicate with each other?",
    "What are the main uses of llamas by humans?",
    "How do llamas react when they feel threatened?",
    "What type of climate is best suited for llamas?",
    "What is the gestation period for a llama?",
    "How often do llamas need to be sheared?",
    "What is a group of llamas called?",
    "How can you tell the age of a llama?",
    "What are some common health issues that llamas face?",
    "How do llamas contribute to their ecosystems?",
    "Can llamas be used as guard animals?",
    "What are the main differences in behavior between wild and domesticated llamas?",
]

alpaca_questions = [
    "What is the average lifespan of an alpaca?",
    "What are the two types of alpacas, and how do they differ?",
    "What is the primary use of alpacas in farming?",
    "Where do alpacas originate from?",
    "What is the typical diet of an alpaca?",
    "How often should alpacas be sheared?",
    "What are some common health issues that alpacas face?",
    "How do alpacas communicate with each other?",
    "What is the gestation period for an alpaca?",
    "How can you distinguish between a male and a female alpaca?",
    "What are the different colors of alpaca fleece?",
    "How does the quality of alpaca fleece compare to sheep's wool?",
    "What are some products made from alpaca fleece?",
    "How much fleece does an alpaca produce annually?",
    "What is the average weight of an adult alpaca?",
    "How do alpacas behave in social settings with other alpacas?",
    "What are some signs of a healthy alpaca?",
    "How can you tell if an alpaca is stressed or unhappy?",
    "What kind of shelter do alpacas need?",
    "How do alpacas contribute to the ecosystem?",
]

cattle_questions = [
    "What are the main differences between beef cattle and dairy cattle?",
    "How long is the gestation period for a cow?",
    "What is the average lifespan of a domesticated cow?",
    "What are the primary breeds of beef cattle?",
    "How much milk does a dairy cow produce on average per day?",
    "What is the purpose of dehorning cattle?",
    "How do farmers typically identify and track their cattle?",
    "What are the common health issues faced by cattle?",
    "How is IVF used in cattle breeding?",
    "What are the nutritional requirements for a growing calf?",
    "What is the significance of the rumen in a cow's digestive system?",
    "How does climate change impact cattle farming?",
    "What are the benefits of rotational grazing for cattle?",
    "How do cattle contribute to greenhouse gas emissions?",
    "What are the key indicators of a cow's health and well-being?",
    "How has the domestication of cattle influenced human agriculture?",
    "What are the ethical considerations in cattle farming?",
    "How do cattle farmers ensure the welfare of their animals during transport?",
    "What are the common methods used for weaning calves from their mothers?",
    "How is biotechnology used to improve cattle breeds and productivity?",
]

plastic_questions = [
    "What does TPU stand for in the context of plastics?",
    "What are the primary uses of TPU plastic in manufacturing?",
    "How does the flexibility of TPU plastic compare to other plastics?",
    "What are the main advantages of using TPU plastic over traditional rubber?",
    "Can TPU plastic be recycled, and if so, how?",
    "What are the key chemical properties of TPU plastic?",
    "How is TPU plastic produced?",
    "What industries commonly use TPU plastic for their products?",
    "How does TPU plastic perform in high-temperature environments?",
    "What are the environmental impacts of producing TPU plastic?",
    "Is TPU plastic considered biodegradable?",
    "How does the hardness of TPU plastic vary with different formulations?",
    "What are some common consumer products made from TPU plastic?",
    "How does the transparency of TPU plastic compare to other thermoplastics?",
    "What are the tensile strength and elongation properties of TPU plastic?",
    "How does TPU plastic react to exposure to UV radiation?",
    "What are the differences between TPU plastic and TPE plastic?",
    "How does TPU plastic handle exposure to chemicals and oils?",
    "What are some limitations or disadvantages of TPU plastic?",
    "How does the cost of TPU plastic compare to other types of plastics?",
]

In [22]:
labels = ["llama", "alpaca", "cattle", "plastic"]
references = [
    llama_statements,
    alpaca_statements,
    cattle_statements,
    plastic_statements,
]
queries = [
        llama_questions,
        alpaca_questions,
        cattle_questions,
        plastic_questions,
]

## Upload to Valor

In [23]:
query_dataset = Dataset.create("queries")
reference_dataset = Dataset.create("references")

model = Model.create("embedding_model")

In [25]:
i = 0
for label, statements in zip(labels, references):
    for statement in statements:
        i += 1
        datum = Datum(uid=f"ref_{i}")
        reference_dataset.add_groundtruth(
            GroundTruth(
                    datum=datum,
                    annotations=[
                        Annotation(
                            labels=[
                                Label(
                                    key="class", 
                                    value=label
                                )
                            ]
                        )
                    ]
                )
            )
        model.add_prediction(
            reference_dataset,
            Prediction(
                datum=datum,
                annotations=[
                    Annotation(
                        embedding=embed(statement)
                    )
                ]
            )
        )

i = 0
for label, questions in zip(labels, queries):
    for question in questions:
        i += 1
        datum = Datum(uid=f"q_{i}")
        query_dataset.add_groundtruth(
            GroundTruth(
                    datum=datum,
                    annotations=[
                        Annotation(
                            labels=[
                                Label(
                                    key="class", 
                                    value=label
                                )
                            ]
                        )
                    ]
                )
            )
        model.add_prediction(
            query_dataset,
            Prediction(
                datum=datum,
                annotations=[
                    Annotation(
                        embedding=embed(question)
                    )
                ]
            )
        )

In [26]:
query_dataset.finalize()
reference_dataset.finalize()

<Response [200]>

# Compute Metrics

In [76]:
def create_dataframe(x: dict, labels: list[str], row_label = "Query", col_label = "Reference"):
    """Creates a pandas dataframe object."""
    col_ix = pd.MultiIndex.from_product([[col_label], labels])
    row_ix = pd.MultiIndex.from_product([[row_label], labels])

    statistic_df = pd.DataFrame(x["value"]["statistics"])
    statistic_df = statistic_df.set_index(row_ix)
    statistic_df.columns = col_ix

    pvalue_df = pd.DataFrame(x["value"]["pvalues"])
    pvalue_df = pvalue_df.set_index(row_ix)
    pvalue_df.columns = col_ix

    return (statistic_df, pvalue_df)

### Document-to-Document

In [77]:
job1 = model.evaluate_embeddings(reference_dataset, metrics_to_return=[MetricType.CramerVonMises])
assert job1.wait_for_completion(timeout=30) == EvaluationStatus.DONE
cvm_doc = job1.metrics[0]

job2 = model.evaluate_embeddings(reference_dataset, metrics_to_return=[MetricType.KolmgorovSmirnov])
assert job2.wait_for_completion(timeout=30) == EvaluationStatus.DONE
ks_doc = job2.metrics[0]

cvm_doc_stats, cvm_doc_pvals = create_dataframe(cvm_doc, labels, row_label="Reference")
ks_doc_stats, ks_doc_pvals = create_dataframe(ks_doc, labels, row_label="Reference")

Cramer-Von Mises Test

In [78]:
cvm_doc_stats

Reference                                 
                        llama     alpaca     cattle    plastic
Reference llama      0.685380   4.245046  12.945897  87.002891
          alpaca     3.647358   0.085400   9.177220  52.701337
          cattle    47.409069  18.203595   0.843800  70.118862
          plastic  192.613619  28.467759  28.045826   0.284175

In [79]:
cvm_doc_pvals

Reference                                          
                          llama        alpaca        cattle       plastic
Reference llama    1.381499e-02  1.294612e-10  4.229290e-09  3.573110e-08
          alpaca   2.846017e-09  6.729539e-01  4.089356e-10  2.298091e-08
          cattle   2.390634e-08  2.781474e-09  5.694686e-03  1.998401e-08
          plastic  5.791465e-08  2.742969e-09  2.139286e-09  1.501549e-01

Kolmgorov-Smirnov Test

In [80]:
ks_doc_stats

Reference                           
                      llama   alpaca    cattle plastic
Reference llama    0.157778  0.39250  0.646667  0.7750
          alpaca   0.156667  0.10000  0.590000  0.6575
          cattle   0.465278  0.74250  0.120000  0.8025
          plastic  0.868889  0.93375  0.875000  0.0500

In [81]:
ks_doc_pvals

Reference                                           
                           llama        alpaca        cattle        plastic
Reference llama     2.636622e-05  2.860665e-13  9.342145e-38  4.359419e-210
          alpaca    1.806079e-11  9.667464e-01  1.614049e-26  8.881348e-110
          cattle   4.619730e-101  3.765331e-44  8.692619e-01  2.647703e-173
          plastic  3.162020e-322  1.152231e-93  5.740524e-76   9.646522e-01

### Query-to-Query

In [82]:
job1 = model.evaluate_embeddings(query_dataset, metrics_to_return=[MetricType.CramerVonMises])
assert job1.wait_for_completion(timeout=30) == EvaluationStatus.DONE
cvm_query = job1.metrics[0]

job2 = model.evaluate_embeddings(query_dataset, metrics_to_return=[MetricType.KolmgorovSmirnov])
assert job2.wait_for_completion(timeout=30) == EvaluationStatus.DONE
ks_query = job2.metrics[0]

# create dataframes
cvm_query_stats, cvm_query_pvals = create_dataframe(cvm_query, labels, col_label="Query")
ks_query_stats, ks_query_pvals = create_dataframe(ks_query, labels, col_label="Query")

Cramer-Von Mises Test

In [83]:
cvm_query_stats

Query                                
                   llama     alpaca     cattle   plastic
Query llama     0.063400  10.148660  18.978225  26.66700
      alpaca   18.192555   0.930600  18.764385  26.65134
      cattle   26.040425  24.119395   0.079000  26.66700
      plastic  26.667000  26.604865  26.621940   0.20780

In [84]:
cvm_query_pvals

Query                                          
                      llama        alpaca        cattle       plastic
Query llama    8.064590e-01  2.439431e-09  5.478722e-09  8.800470e-09
      alpaca   2.753599e-09  3.538873e-03  4.568793e-09  8.721066e-09
      cattle   6.072758e-09  1.730585e-09  7.101283e-01  8.800470e-09
      plastic  8.800470e-09  8.489087e-09  8.573683e-09  2.555255e-01

Kolmgorov-Smirnov Test

In [85]:
ks_query_stats

Query                        
                llama  alpaca  cattle plastic
Query llama    0.1800  0.5900  0.7225  1.0000
      alpaca   0.7675  0.2200  0.7250  0.9975
      cattle   0.9850  0.9000  0.2800  1.0000
      plastic  1.0000  0.9975  0.9975  0.1400

In [86]:
ks_query_pvals

Query                                             
                       llama         alpaca         cattle        plastic
Query llama     3.959399e-01   1.614049e-26   1.887124e-41  9.795786e-108
      alpaca    9.994633e-48   1.785867e-01   8.824338e-42  9.893744e-106
      cattle    1.673017e-98   6.781257e-72   3.919459e-02  9.795786e-108
      plastic  9.795786e-108  9.893744e-106  9.893744e-106   7.166468e-01

### Query-to-Document

In [87]:
job1 = model.evaluate_embeddings(query_dataset, metrics_to_return=[MetricType.CramerVonMises])
assert job1.wait_for_completion(timeout=30) == EvaluationStatus.DONE
cvm_query_doc = job1.metrics[0]

job2 = model.evaluate_embeddings(query_dataset, metrics_to_return=[MetricType.KolmgorovSmirnov])
assert job2.wait_for_completion(timeout=30) == EvaluationStatus.DONE
ks_query_doc = job2.metrics[0]

# create dataframes
cvm_query_doc_stats, cvm_query_doc_pvals = create_dataframe(cvm_query, labels)
ks_query_doc_stats, ks_query_doc_pvals = create_dataframe(ks_query, labels)

Cramer-Von Mises Test

In [88]:
cvm_query_doc_stats

Query                                
                   llama     alpaca     cattle   plastic
Query llama     0.063400  10.148660  18.978225  26.66700
      alpaca   18.192555   0.930600  18.764385  26.65134
      cattle   26.040425  24.119395   0.079000  26.66700
      plastic  26.667000  26.604865  26.621940   0.20780

In [89]:
cvm_query_doc_pvals

Query                                          
                      llama        alpaca        cattle       plastic
Query llama    8.064590e-01  2.439431e-09  5.478722e-09  8.800470e-09
      alpaca   2.753599e-09  3.538873e-03  4.568793e-09  8.721066e-09
      cattle   6.072758e-09  1.730585e-09  7.101283e-01  8.800470e-09
      plastic  8.800470e-09  8.489087e-09  8.573683e-09  2.555255e-01

Kolmgorov-Smirnov Test

In [90]:
ks_query_doc_stats

Query                        
                llama  alpaca  cattle plastic
Query llama    0.1800  0.5900  0.7225  1.0000
      alpaca   0.7675  0.2200  0.7250  0.9975
      cattle   0.9850  0.9000  0.2800  1.0000
      plastic  1.0000  0.9975  0.9975  0.1400

In [91]:
ks_query_doc_pvals

Query                                             
                       llama         alpaca         cattle        plastic
Query llama     3.959399e-01   1.614049e-26   1.887124e-41  9.795786e-108
      alpaca    9.994633e-48   1.785867e-01   8.824338e-42  9.893744e-106
      cattle    1.673017e-98   6.781257e-72   3.919459e-02  9.795786e-108
      plastic  9.795786e-108  9.893744e-106  9.893744e-106   7.166468e-01